This notebooks shows how to use code interpreter [OpenAI assistant](https://platform.openai.com/docs/assistants/tools/code-interpreter). This tool generates the code and executes the code in the sandbox.
- steps:
  - create an assistant (LLM with code interpreter)
  - call the assistant to write the code, and executes the code in the sandbox (this might contain multiple rounds in order to get the answer)
  - get the answer from the assistant

In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import os
from jinja2 import Environment, FileSystemLoader
from pathlib import Path
import time
import sys
from PIL import Image

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.utils import convert_types, eval
from utils.vars import DATA_DIR, DATASET_FILES, QUESTION_FILES
from utils.assistants import AzureOpenAIAssistant

load_dotenv()
ASSISTANT_NAME_PREFIX = "code_interpreter"

In [ ]:
# get the prompt template
prompt_path = "prompts/prompt.jinja2"

# get the client object
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2025-03-01-preview",  # different from assistant
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)
assistant = AzureOpenAIAssistant(client=client)

In [ ]:
display(assistant.list_all_assistants())

In [ ]:
# delete assistant
assistant.delete_assistant(name=f"{ASSISTANT_NAME_PREFIX}_nyc_taxi")
assistant.delete_assistant(name=f"{ASSISTANT_NAME_PREFIX}_melbourne_temp")
assistant.delete_assistant(name=f"{ASSISTANT_NAME_PREFIX}_air_passengers")

display(assistant.list_all_assistants())

In [ ]:
# read the prompt
instruction = (
    Environment(loader=FileSystemLoader(".")).get_template(prompt_path).render()
)

df_result = []

# loop through files
# loop over the question files
for question_path in QUESTION_FILES:
    question_path = Path(question_path)
    print(f"Question file: {question_path.name}")
    # read questions
    df_questions = pd.read_csv(DATA_DIR / question_path)
    # loop through each csv file
    for dataset_path in DATASET_FILES:
        dataset_path = Path(dataset_path)
        print(f"file: {dataset_path.name}")

        # check if file has been uploaded to the client
        file_id = assistant.upload_or_retrieve_file(file_path=dataset_path)

        # create or retrieve an assistant
        assistant_id = assistant.create_or_retrieve(
            prompt_path=prompt_path,
            assistant_name=f"{ASSISTANT_NAME_PREFIX}_{dataset_path.stem}",
            tools=[{"type": "code_interpreter"}],
            tool_resources={"code_interpreter": {"file_ids": [file_id]}},
        )

        # loop through questions
        for _, row in df_questions.iterrows():
            question = row["question"]
            answer_true = row[dataset_path.name]

            start_time = time.time()

            result = assistant.ask_a_question(
                question=question, assistant_id=assistant_id
            )

            df_result.append(
                {
                    **result,
                    "question_file": question_path.name,
                    "question": question,
                    "execution_time_s": round(time.time() - start_time, 2),
                    "dataset_file": dataset_path.name,
                    "org_answer_true": answer_true,
                }
            )

In [ ]:
# eval
df_result = pd.DataFrame(df_result)

df_result["answer_pred"] = df_result["org_answer_pred"].apply(
    lambda x: convert_types(x)
)
df_result["answer_true"] = df_result["org_answer_true"].apply(
    lambda x: convert_types(x)
)

# loop through each file
eval(df=df_result, details=True)

In [ ]:
# check detailed observability metrics
df_result.groupby(["question_file", "dataset_file"])[
    ["total_tokens", "execution_time_s"]
].describe()[[("total_tokens", "mean"), ("execution_time_s", "mean")]].round(2)

In [ ]:
# check overall observability metrics
df_result.describe().round(2)

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_result.groupby(["dataset_file"]).describe())

## Run one question

In [ ]:
# list all assistants
assistant.list_all_assistants()

In [ ]:
question = "What is the target column?"
assistant_id = "asst_VGmOogPrbDPXHIJeUbVmWWzZ"
print(
    f'answer_true: {df_questions[df_questions["question"] == question]["nyc_taxi.csv"].values[0]}'
)
result = assistant.ask_a_question(question=question, assistant_id=assistant_id)
result

## Appendix: Generate an image

In [ ]:
prompt_path = "prompts/draw_picture.jinja2"
question = "Generate a box plot of the target column using seaborn with text annotation for min, max, q1, q3, and median."
file_path = Path(DATA_DIR) / "nyc_taxi.csv"
file_id = assistant.upload_or_retrieve_file(file_path=file_path)

In [ ]:
assistant_id = assistant.create_or_retrieve(
    assistant_name=f"{ASSISTANT_NAME_PREFIX}_{file_path.stem}_plot",
    prompt_path=prompt_path,
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file_id]}},
)
result = assistant.ask_a_question(question=question, assistant_id=assistant_id)

# print the output
print(f'messages: {"\n".join(result["messages"])}')
print(f'steps: {result["steps"]}')

In [ ]:
img = result["attachments"][0]
with open(img["file_name"], "wb") as file:
    file.write(img["file_bytes"])
Image(filename=img["file_name"], width=1000)